### Convert EZMT pre-processed NHDv2 Plus data to available water per technology

In [2]:
import os
import glob

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd


In [3]:
data_dir = '/Users/d3y010/projects/cerf/data/ezmt/surface_water_flow_nhdplus_v2_erom_eispc_v2'

water_shp_dir = '/Users/d3y010/projects/cerf/data/suitability/nhd2plus'

output_dir = '/Users/d3y010/projects/cerf/data/suitability/GRIDCERF/Tech_Specific_Layers'

template_raster = os.path.join(output_dir, 'cerf_railnodes5km_navwaters5km_conus.tif')

f = os.path.join(data_dir, 'ez_gis.surface_water_flow_nhdplus_v2_erom_eispc_v2.shp')


In [3]:
# get cerf CRS from template raster
with rasterio.open(template_raster) as src:
    target_crs = src.crs
    

In [9]:
%%time 

# only keep gallons per minute flow and geometry and reproject
gdf = gpd.read_file(f)[['q_gpm', 'geometry']].to_crs(target_crs)

# convert to millions gallons per day
gdf['mgd'] = (gdf['q_gpm'] / 1000000) * 60 * 24

# drop gpm field
gdf.drop(columns=['q_gpm'], inplace=True)

# set raster value
gdf['value'] = 0

gdf.mgd.describe()


CPU times: user 51.5 s, sys: 1.42 s, total: 52.9 s
Wall time: 53 s


count    346016.000000
mean       1763.519917
std       13989.819998
min          28.800465
25%          54.261967
50%         129.799854
75%         478.966720
max      484351.359744
Name: mgd, dtype: float64

### CERF bins for minimum mean annual flow requirements

In [5]:
bins = {'cerf_nhd2plus_surfaceflow_greaterthan145mgd_buffer20km': 145,
        'cerf_nhd2plus_surfaceflow_greaterthan120mgd_buffer20km': 120,
        'cerf_nhd2plus_surfaceflow_greaterthan110mgd_buffer20km': 110,
        'cerf_nhd2plus_surfaceflow_greaterthan95mgd_buffer20km': 95,
        'cerf_nhd2plus_surfaceflow_greaterthan75mgd_buffer20km': 75,
        'cerf_nhd2plus_surfaceflow_greaterthan70mgd_buffer20km': 70,
        'cerf_nhd2plus_surfaceflow_greaterthan55mgd_buffer20km': 55,
        'cerf_nhd2plus_surfaceflow_greaterthan40mgd_buffer20km': 40,
        'cerf_nhd2plus_surfaceflow_greaterthan35mgd_buffer20km': 35,
        'cerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km': 25,
        'cerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km': 10,
        'cerf_nhd2plus_surfaceflow_greaterthan2mgd_buffer20km': 2}


### Create a buffered shapefile of flowlines matching the flow requirement

In [10]:
%%time

for i in bins.keys():
    
    print(i)
    
    # extract the flowlines that support the minimum flow requirement
    gdx = gdf.loc[gdf['mgd'] > bins[i]].copy()

    # buffer by 20 km (20000 meters)
    gdx['geometry'] = gdx.buffer(20000)
    
    output_shp = os.path.join(water_shp_dir, f"{i}.shp")
    
    gdx[['value', 'geometry']].to_file(output_shp)



cerf_nhd2plus_surfaceflow_greaterthan145mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan120mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan110mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan95mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan75mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan70mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan55mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan40mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan35mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km
cerf_nhd2plus_surfaceflow_greaterthan2mgd_buffer20km
CPU times: user 26min 5s, sys: 29.1 s, total: 26min 35s
Wall time: 26min 41s


### Rasterize each shapefile into CERF's spatial requirements

In [4]:
%%time 

shp_list = glob.glob(os.path.join(water_shp_dir, 'cerf_nhd2plus_surfaceflow_greaterthan*mgd_buffer20km.shp'))

for i in shp_list:
        
    basename = os.path.basename(i)
    base_noext = os.path.splitext(basename)[0]
    
    output_raster = os.path.join(output_dir, f"{base_noext}.tif")

    gdal_rasterize_cmd = f"gdal_rasterize -l {base_noext} -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {i} {output_raster}"

    print(gdal_rasterize_cmd)
    
    os.system(gdal_rasterize_cmd)


gdal_rasterize -l cerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/projects/cerf/data/suitability/nhd2plus/cerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km.shp /Users/d3y010/projects/cerf/data/suitability/GRIDCERF/Tech_Specific_Layers/cerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km.tif
gdal_rasterize -l cerf_nhd2plus_surfaceflow_greaterthan110mgd_buffer20km -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/projects/cerf/data/suitability/nhd2plus/cerf_nhd2plus_surfaceflow_greaterthan110mgd_buffer20km.shp /Users/d3y010/projects/cerf/data/suitability/GRIDCERF/Tech_Specific_Layers/cerf_nhd2plus_surfaceflow_greaterthan110mgd_buffer20km.tif
gdal_rasterize -l cerf_nhd2plus_surfaceflow_greaterthan120mgd_buffer20km -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389